In [1]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv('epi_r.csv')

In [3]:
df.head()

,title,rating,calories,protein,fat,sodium,#cakeweek,#wasteless,22-minute meals,3-ingredient recipes,...,yellow squash,yogurt,yonkers,yuca,zucchini,cookbooks,leftovers,snack,snack week,turkey
0,"Lentil, Apple, and Turkey Wrap",2.500,426.0,30.0,7.0,559.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,Boudin Blanc Terrine with Red Onion Confit,4.375,403.0,18.0,23.0,1439.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Potato and Fennel Soup Hodge,3.750,165.0,6.0,7.0,165.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Mahi-Mahi in Tomato Olive Sauce,5.000,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Spinach Noodle Casserole,3.125,547.0,20.0,32.0,452.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
null_count = df.isnull().sum()

In [17]:
null_count[null_count>0]

calories    4117
protein     4162
fat         4183
sodium      4119
dtype: int64

In [18]:
recipes = df.dropna()

In [19]:
recipes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15864 entries, 0 to 20051
Columns: 680 entries, title to turkey
dtypes: float64(679), object(1)
memory usage: 82.4+ MB


Removed the null values in the dietary information column. Will run to see how it performs

In [31]:
from sklearn.svm import SVR
svr = SVR()
X = recipes.drop(['rating','title'], 1)
Y = recipes['rating']

In [32]:
svr.fit(X,Y)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [33]:
svr.score(X, Y)

0.20120576501871634

Pretty good improvement, from 3.8% to 20%. The other two ideas are to clean up the data further and possibly transform the target variable into binary, to make it a classification problem.

In [38]:
recipes.corr()['rating'].nlargest(50)

rating               1.000000
bon appétit          0.129069
peanut free          0.078845
soy free             0.078212
roast                0.071724
bake                 0.070332
tree nut free        0.069766
fall                 0.065290
sauté                0.061349
thanksgiving         0.060912
winter               0.058439
grill/barbecue       0.048829
dinner               0.047193
cheese               0.044035
onion                0.043335
low carb             0.042854
braise               0.042613
beef                 0.041673
goat cheese          0.040612
tomato               0.040270
kosher               0.039722
herb                 0.038745
no sugar added       0.037998
christmas            0.037468
pork                 0.037346
lamb                 0.037263
broil                0.037252
marinate             0.037096
pescatarian          0.036636
mixer                0.036276
sugar conscious      0.036155
gourmet              0.034671
fruit                0.034056
backyard b

In [41]:
from sklearn.preprocessing import StandardScaler
x_standard = StandardScaler().fit_transform(X)

In [60]:
from sklearn.decomposition import PCA

pca = PCA(n_components=10)

principalComponents = pca.fit_transform(x_standard)

principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4', 'principal component 5','principal component 6', 'principal component 7', 'principal component 8', 'principal component 9', 'principal component 10'])

In [61]:
finalDf = pd.concat([principalDf, recipes['rating']], axis = 1)

In [62]:
finalDf.head()

,principal component 1,principal component 2,principal component 3,principal component 4,principal component 5,principal component 6,principal component 7,principal component 8,principal component 9,principal component 10,rating
0,1.954550,-1.448963,-1.079739,-0.100166,0.221248,-0.053230,0.737713,-2.162105,-0.109865,-0.666614,2.500
1,2.185498,1.215606,-1.349805,0.905761,-2.135310,0.368743,-1.261685,-0.252990,-1.537613,0.544743,4.375
2,2.058710,-1.013103,-0.880722,0.141684,-0.760709,-0.208732,1.328259,1.172000,-0.367689,-0.521092,3.750
3,0.799799,-0.599464,-1.799181,0.157899,-1.161381,-0.237098,1.870164,-1.634783,-0.718197,-0.165604,NaN
4,2.090818,-1.514577,-0.609025,-1.127640,1.941365,0.748605,1.757536,-1.946074,-0.874564,-0.234891,3.125


In [63]:
finalDf.dropna(inplace=True)

In [64]:
svr_pca = SVR()
X_pca = finalDf.drop('rating', 1)
Y_pca = finalDf['rating']

In [65]:
svr_pca.fit(X_pca,Y_pca)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [66]:
svr_pca.score(X_pca,Y_pca)

0.02591708001922755

that didnt work out so well... Will try to run the model with the top 30 correlation features vs the target

In [80]:
top30 = list(recipes.corr()['rating'].nlargest(31).index)[1:]

In [92]:
recipes[top30]

,bon appétit,peanut free,soy free,roast,bake,tree nut free,fall,sauté,thanksgiving,winter,...,herb,no sugar added,christmas,pork,lamb,broil,marinate,pescatarian,mixer,sugar conscious
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
10,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
13,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [95]:
recipes['binary_rating'] = np.where(recipes['rating']>=4,1,0)

/home/diegofvargas/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [96]:
recipes['binary_rating']

0        0
1        1
2        0
4        0
5        1
8        1
9        0
10       0
12       1
13       1
14       0
15       1
16       0
17       1
18       1
19       1
20       1
24       0
25       0
26       0
27       0
28       1
29       1
32       1
33       0
35       0
37       1
38       1
39       0
40       1
        ..
20019    0
20020    0
20021    1
20022    1
20023    0
20024    1
20025    1
20027    1
20028    0
20029    0
20030    0
20031    1
20032    1
20033    1
20034    1
20036    1
20037    0
20038    0
20039    0
20040    1
20041    0
20043    1
20044    0
20045    0
20046    0
20047    0
20048    1
20049    1
20050    1
20051    1
Name: binary_rating, Length: 15864, dtype: int64

In [98]:
from sklearn.svm import SVC
svc = SVC()
X = recipes[top30]
Y = recipes['binary_rating']

In [99]:
svc.fit(X,Y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [100]:
svc.score(X, Y)

0.5747604639435199

Much better accuracy!